In [2]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs236299-2023-spring/lab2-4.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [3]:
# Initialize Otter
import otter
grader = otter.Notebook()

$$
\renewcommand{\vect}[1]{\mathbf{#1}}
\renewcommand{\cnt}[1]{\sharp(#1)}
\renewcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\renewcommand{\softmax}{\operatorname{softmax}}
\renewcommand{\Prob}{\Pr}
\renewcommand{\given}{\,|\,}
$$

# Course 236299
## Lab 2-4 – Sequence labeling with hidden Markov models

In [4]:
import math
from collections import defaultdict

# Introduction

Hidden Markov models (HMM) are a fundamental generative method for sequence labeling NLP tasks such as part-of-speech tagging (as in the present lab) and information extraction (as in the second project segment). In this lab, you'll train, apply, and evaluate some simple sequence labeling algorithms culminating in HMM.

To keep things manageable, the dataset you'll use will involve very few word types, only six (plus a special beginning of sentence token), but these are quite ambiguous with regard to part of speech. We'll use the following labels for parts of speech:

In [5]:
parts_of_speech = [
    "<bos>", # beginning of sentence marker
    "N",     # noun
    "V",     # main verb
    "M",     # modal verb
    "P",     # preposition
    "A",     # adjective
    "R"      # adverb
]

The vocabulary of word types, along with their possible parts of speech, is given by the following dictionary:

In [6]:
vocabulary = {
    "<bos>":   ["<bos>"],
    "can":     ["N", "V", "M"],
    "canned":  ["A", "V"],
    "canners": ["N"],
    "fish":    ["N", "V"],
    "for":     ["P"],
    "not":     ["R"]
}

Here are a few sentences constructed with these words:

In [7]:
text = """
    <bos> canners canned fish
    <bos> can canners can fish
    <bos> fish can not fish
    <bos> can fish can fish can
    <bos> canners fish fish for can
    <bos> canners can fish for fish
    <bos> canners fish for fish
    <bos> fish can canned fish
    <bos> canners can not can canned fish
    <bos> fish can can fish for canners
"""

and the corresponding POS sequences, for the first few sentences. Complete the rest.
<!--
BEGIN QUESTION
name: text_pos
-->

In [8]:
#TODO -- Provide part-of-speech sequences in this format for the rest of the sample sentences.
text_pos = """
    <bos> N V N 
    <bos> M N V N
    <bos> N M R V
    <bos> M N V N N
    <bos> N V N P N
    <bos> N M V P N
    <bos> N V P N
    <bos> N N A N
    <bos> N M R V A N
    <bos> N M V N P N
"""

text_pos ="""
    <bos> N V N
    <bos> M N V N
    <bos> N M R V
    <bos> M N V N N
    <bos> N V N P N
    <bos> N M V P N
    <bos> N V P N
    <bos> N V A N
    <bos> N M R V A N
    <bos> N M V N P N
"""

In [9]:
grader.check("text_pos")

All tests passed!

> Just to make sure we're all on the same page -- and because you'll need the right tagging to calculate some probabilities below -- you can find our intended solution to this question at <https://pastebin.com/raw/Us8UcWbn>. You should check your solution before moving on.

We tokenize the sentences and label the tokens with their POS.

In [10]:
def tokenize(text):
  result = []
  for line in text.strip().split("\n"):
    result.append([item for item in line.strip().split()])
  return result

tagged_text = [list(zip(sentence, poses))
                 for sentence, poses 
                   in zip(tokenize(text), tokenize(text_pos))]

Here are a couple of examples to indicate what the tagged sentences look like.

In [11]:
print(tagged_text[0])
print(tagged_text[1])

[('<bos>', '<bos>'), ('canners', 'N'), ('canned', 'V'), ('fish', 'N')]
[('<bos>', '<bos>'), ('can', 'M'), ('canners', 'N'), ('can', 'V'), ('fish', 'N')]


For reference, here is a table showing the frequency distribution for each word type and each part of speech it can be used as.

In [12]:
counts = defaultdict(lambda: defaultdict(int))
for sentence in tagged_text:
  for type, pos in sentence:
    counts[type][pos] += 1

print(f'{"TYPE":8} {"POS":6} {"COUNT"}')
for type, type_counts in counts.items():
  for pos, count in type_counts.items():
    print(f'{type:8} {pos:6} {count:2}')

TYPE     POS    COUNT
<bos>    <bos>  10
canners  N       7
canned   V       1
canned   A       2
fish     N      13
fish     V       4
can      M       6
can      V       5
can      N       2
not      R       2
for      P       4


# Majority label
The first sequence labeling method we'll use is simply to choose for each word the POS label it most frequently occurs as in the training data. The table above provides the required information directly.

Choosing the majority label for a word sequence $\vect{w} = \langle{w_1, w_2, \ldots, w_m}\rangle$ is tantamount to maximizing the probability of the label sequence assuming independence of the label conditioned on the word, that is, selecting the tag sequence $\vect{t} = \langle{t_1, t_2, \ldots, t_m}\rangle$ given by
$$ \argmax{\vect{t}} \prod_{i=1}^m \Prob(t_i \given w_i) $$

How would the majority label method label the following test sentence (which we've marked with the words' correct ("gold") parts of speech)?

> &lt;bos&gt;[&lt;bos&gt;] canners[N] can[V] canned[A] fish[N]

Give your answer in the next cell in the form of a list of strings for the POS labels.
<!--
BEGIN QUESTION
name: example_majority_labeling
-->

In [13]:
#TODO
# "<bos> canners can canned fish"
example_majority_labeling = ["<bos>", "N", "M", "A", "N"]

In [14]:
grader.check("example_majority_labeling")

All tests passed!

By inspection, what is the accuracy of the majority labeling, given as a proportion of the words in the sentence (including the beginning of sentence token)?
<!--
BEGIN QUESTION
name: example_maj_label_accuracy
-->

In [15]:
#TODO
example_maj_label_accuracy = 4/5

In [16]:
grader.check("example_maj_label_accuracy")

All tests passed!

# Majority bigram labeling

It may occur to you that what part of speech a word has _depends on its context_. Suppose we relax the assumption that tag probabilities depend only on the word being tagged, and condition them on the previous word as well. (For the first word in the sentence, we'll condition on that fact, by conditioning it on the special start token.) In summary, we'll condition on the bigram that ends at the word being tagged:
$$ \argmax{\vect{t}} \prod_{i=1}^m \Prob(t_i \given w_{i-1} w_i) $$

What is the majority bigram labeling of the test sentence? Again, give your answer in the form of a list of strings for the POS labels.
<!--
BEGIN QUESTION
name: example_majority_bigram_labeling
-->

In [17]:
counts = defaultdict(lambda: defaultdict(int))
prev = []
for sentence in tagged_text:
    for type, pos in sentence:
        prev = prev+[type]
        bigram = tuple(prev[-2:])
        counts[bigram][pos] += 1
    prev = []

print(f'{"TYPE":22} {"POS":6} {"COUNT"}')
for type, type_counts in counts.items():
  for pos, count in type_counts.items():
    print(f'{str(type):22} {pos:6} {count:2}')

TYPE                   POS    COUNT
('<bos>',)             <bos>  10
('<bos>', 'canners')   N       5
('canners', 'canned')  V       1
('canned', 'fish')     N       3
('<bos>', 'can')       M       2
('can', 'canners')     N       1
('canners', 'can')     V       1
('canners', 'can')     M       2
('can', 'fish')        N       4
('can', 'fish')        V       1
('<bos>', 'fish')      N       3
('fish', 'can')        M       2
('fish', 'can')        V       2
('fish', 'can')        N       1
('can', 'not')         R       2
('not', 'fish')        V       1
('canners', 'fish')    V       2
('fish', 'fish')       N       1
('fish', 'for')        P       4
('for', 'can')         N       1
('for', 'fish')        N       2
('can', 'canned')      A       2
('not', 'can')         V       1
('can', 'can')         V       1
('for', 'canners')     N       1


In [18]:
#TODO
# "<bos> canners can canned fish"
example_majority_bigram_labeling = ["<bos>", "N", "M", "A", "N"]

In [19]:
grader.check("example_majority_bigram_labeling")

All tests passed!

By inspection, what is the accuracy of the majority bigram labeling, given as a proportion of the words?
<!--
BEGIN QUESTION
name: example_maj_bigram_label_accuracy
-->

In [20]:
#TODO
example_maj_bigram_label_accuracy = 4/5

In [21]:
grader.check("example_maj_bigram_label_accuracy")

All tests passed!

# Hidden Markov models

Now we get to the real point, using an HMM model. Recall that in an HMM model, we assume that the joint tag/word sequence is generated by 

1. Selecting a tag sequence according to a Markov model whose states correspond to tags and whose transitions from state $t_i$ to $t_j$ are governed by a _transition probability_ $a_{ij} = \Prob(t_i \rightarrow t_j)$, and then
2. Selecting a word sequence from the tag sequence where for tag $t_i$ we observe word $x_i$ of type $w_j$ governed by an _emission probability_ $b_{i}(w_j) = \Prob(t_i \rightarrow w_j)$.

> Here, we're using a notation $\Prob(t_i \rightarrow t_j)$ to indicate the probability that a word tagged $t_i$ is followed by a word tagged $t_j$ and $\Prob(t_i \rightarrow w_j)$ to indicate the probability that a word tagged $t_i$ is the word $w_j$.

## Estimating the transition and emission probabilities

We estimate these transition and emission probabilities by looking at the empirical probabilities in the training data, counting and perhaps smoothing as usual. That is, for the (unsmoothed) transition probabilities, we estimate
$$ a_{ij} \approx \frac{\cnt{t_i \rightarrow t_j}}{\sum_k \cnt{t_i \rightarrow t_k}} $$
and for the emission probabilities
$$ b_i(w_j) \approx \frac{\cnt{t_i \rightarrow w_j}}{\cnt{t_i}} $$

For instance, we note that there are 4 times in the training data where the tag $N$ is followed by the tag $M$, out of the 21 occurrences of the tag $N$. Thus, we estimate the corresponding transition probability $a_{NM} \approx 4/21$.


Similarly, the emission probability $b_M(can)$ for tag $M$ generating the word $can$ is $6/6 = 1$, since every occurrence of the tag $M$ corresponds to the word $can$ in the training data.

For your convenience, we've computed and provided full tables for the transition and emission probabilities below.

In [22]:
# Generate counts
bigram_tag_counts = defaultdict(lambda: defaultdict(int))
unigram_tag_counts = defaultdict(int)
tag_word_counts = defaultdict(lambda: defaultdict(int))
tag_counts = defaultdict(int)

for sentence in tagged_text:
    for (w1, t1), (w2, t2) in list(zip(sentence[:-1], sentence[1:])):
        bigram_tag_counts[t1][t2] += 1
        unigram_tag_counts[t1] += 1
    for w, t in sentence:
        tag_word_counts[t][w] += 1
        tag_counts[t] += 1

# Generate transition and emission probabilities
a = defaultdict(lambda: defaultdict(int))
b = defaultdict(lambda: defaultdict(int))

for t1 in parts_of_speech:
    for t2 in parts_of_speech:
        a[t1][t2] = bigram_tag_counts[t1][t2] / unigram_tag_counts[t1]
    for w1 in vocabulary.keys():
        b[t1][w1] = tag_word_counts[t1][w1] / tag_counts[t1]

# Print tables of probabilities

print("Transition probabilities: a_ij")
print(f"{' ':6}", end="")
for t in parts_of_speech:
    print(f"{t:>6}", end="")
print()
for t1 in parts_of_speech:
    print(f"{t1:<6}", end="")
    for t2 in parts_of_speech:
        print(f"{a[t1][t2]:>6.2f}", end="")
    print("")
 
print("\nEmission probabilities: b_i(w_j)")
print(f"{' ':6}", end="")
for w in vocabulary.keys():
    print(f"{w:>8}", end="")
print()
for t in parts_of_speech:
    print(f"{t:<6}", end="")
    for w in vocabulary.keys():
        print(f"{b[t][w]:>8.2f}", end="")
    print()

Transition probabilities: a_ij
       <bos>     N     V     M     P     A     R
<bos>   0.00  0.80  0.00  0.20  0.00  0.00  0.00
N       0.00  0.08  0.46  0.31  0.15  0.00  0.00
V       0.00  0.56  0.00  0.00  0.22  0.22  0.00
M       0.00  0.33  0.33  0.00  0.00  0.00  0.33
P       0.00  1.00  0.00  0.00  0.00  0.00  0.00
A       0.00  1.00  0.00  0.00  0.00  0.00  0.00
R       0.00  0.00  1.00  0.00  0.00  0.00  0.00

Emission probabilities: b_i(w_j)
         <bos>     can  canned canners    fish     for     not
<bos>     1.00    0.00    0.00    0.00    0.00    0.00    0.00
N         0.00    0.09    0.00    0.32    0.59    0.00    0.00
V         0.00    0.50    0.10    0.00    0.40    0.00    0.00
M         0.00    1.00    0.00    0.00    0.00    0.00    0.00
P         0.00    0.00    0.00    0.00    0.00    1.00    0.00
A         0.00    0.00    1.00    0.00    0.00    0.00    0.00
R         0.00    0.00    0.00    0.00    0.00    0.00    1.00


## An example HMM trellis

<img src="https://github.com/nlp-course/data/raw/master/Resources/hmm-figure.png" width="75%" align=right />

Now consider the HMM generating the example sentence "canners can canned fish". The figure at right contains the _trellis_ for the sentence. The horizontal axis corresponds to the words in the sentence, one at a time. The vertical axis corresponds to the states of the HMM (that is, the parts of speech). The gray arrows that connect a tag on the left to a tag on the right correspond to the transition probabilities. The red arrows that connect a tag to a word directly below correspond to the emission probabilities. 

For convenient reference, we've labeled the rows with letters and the columns with numbers (in teal!) so that particular nodes can be referred to as, for example, B1.

(The red lines are intended to go from a state node to the word node directly below it; for instance, the red line immediately below B1 goes to the word node F1 at the bottom of the figure. Some of those lines are depicted with a crossbar to indicate that they are running "underneath" other graphic objects in the figure.)

We've highlighted two paths through the trellis from the beginning to the end of the sentence, corresponding to different taggings of the sentence:

1. A0-B1-C2-E3-B4
2. A0-B1-D2-C3-B4

Answer the following questions about this trellis.

What is the path through the trellis corresponding to the _majority bigram labeling_ that you determined above? Give your answer as a string, like the path examples in the previous cell.
<!--
BEGIN QUESTION
name: majority_path
-->

In [23]:
#["<bos>", "N", "M", "A", "N"]
majority_path = "A0-B1-D2-E3-B4"

In [24]:
grader.check("majority_path")

All tests passed!

The probability of a path is just the product of the probabilities of all the transitions along the path and all the emissions from nodes in the path to observed words. Use the tables above to calculate the probability of the first highlighted path (A0-B1-C2-E3-B4) by multiplying together the appropriate probabilities. Don't forget the emission probabilities, corresponding to the edges A0-F0, B1-F1, C2-F2, E3-F3, and B4-F4.
<!--
BEGIN QUESTION
name: highlight1_path_probability
-->

In [25]:
#TODO
# A0-B1-C2-E3-B4
transitions = (0.8)*(0.36)*(0.12)*(1)
emisions = (1)*(0.3)*(0.44)*(1)*(0.57)
highlight1_path_probability = transitions * emisions
highlight1_path_probability

0.0026002943999999997

In [26]:
grader.check("highlight1_path_probability")

All tests passed!

Do the same for the second highlighted path (A0-B1-D2-C3-B4).
<!--
BEGIN QUESTION
name: highlight2_path_probability
-->

In [27]:
#TODO
transitions = (0.8) * (0.29) * (0.33) * (0.62)
emisions = (1) * (0.3) * (1) * (0.11) * (0.62)
highlight2_path_probability = transitions * emisions
highlight2_path_probability

0.0009711789120000001

In [28]:
grader.check("highlight2_path_probability")

All tests passed!

These two paths turn out to be the two paths through the trellis with the highest probabilities. (You'll have to trust us.) Based on that fact, which tagging has the highest probability according to this HMM? Give your solution in the same format as you did for `example_majority_labeling` above.
<!--
BEGIN QUESTION
name: example_highest_labeling
-->

In [29]:
#TODO
example_highest_labeling = ["<bos>", "N", "V", "A", "N"]

In [30]:
grader.check("example_highest_labeling")

All tests passed!

By inspection, what is the accuracy of the highest probability HMM labeling, given as a proportion of the words?
<!--
BEGIN QUESTION
name: example_highest_label_accuracy
-->

In [31]:
#TODO
example_highest_label_accuracy = 5/5

In [32]:
grader.check("example_highest_label_accuracy")

All tests passed!

Now, recall the majority bigram labeling, and the path for it that you developed above. What is the probability of that path according to the HMM?

<!--
BEGIN QUESTION
name: majority_path_probability
-->

In [33]:
transitions = (0.8) * (0.29) * (0) * (1)
emisions = (1) * (0.3) * (1) * (1) * (0.57)
majority_path_probability = transitions*emisions

In [34]:
grader.check("majority_path_probability")

All tests passed!

## Calculating the highest probability tagging - The Viterbi algorithm

Above, we merely asserted that the two highlighted paths are the two most probable, so that it was a simple matter to find the highest probability tagging by just comparing the probabilities of those two. But in general there can be a huge number of paths through a trellis such as this.

<!-- BEGIN QUESTION -->

**Question:** If there are $N$ tags and a sentence of length $M$, how many paths through the HMM trellis will there be (using big-$O$ notation)?

<!--
BEGIN QUESTION
name: open_response_how_many_paths
manual: true
-->

$O(N^M)$


<!-- END QUESTION -->



The Viterbi algorithm, named after famed electrical engineer [Andrew Viterbi](https://en.wikipedia.org/wiki/Andrew_Viterbi), is an efficient dynamic programming algorithm for performing this (otherwise impractical) computation. We'll do the first few steps of the Viterbi algorithm for the example here.

Given a string of words $\vect{x} = \langle w_0, w_1, \ldots, w_M \rangle$ and a set of states (tags) $\vect{q} = \{q_0, q_1, \ldots, q_N\}$, the algorithm works by calculating a series of values $v_i(j)$ where $i$ ranges over the words in the sentence from $1$ to $M$ and $j$ ranges over the tags from $1$ to $N$. For simplicity, we'll assume an extra word and tag at the beginning of the sentence, as above, so $w_0 = \texttt{<bos>}$ and $q_0 = \texttt{<bos>}$.

The values $v_i(j)$ correspond to the probability of the best (highest probability) path starting in state 0, emitting $w_0, \ldots, w_i$, and ending in state $q_j$. The definition for $v$ then is:

\begin{align*}
  v_0(0) &= 1 \\
  v_0(j) &= 0  &\mbox{for $j > 0$} \\
  v_i(j) &= \max_{j'=0}^N v_{i-1}(j') \cdot a_{j' j} \cdot b_{j}(w_i)
         & \mbox{for $i > 0$}
\end{align*}

In addition, we'll want to keep track of which states the best paths go through, so in addition to tracking the best probability with $v_i(j)$, we'll track the immediately preceding state that led to that best path -- the "back pointer" -- with $bp_i(j)$:

\begin{align*}
  bp_i(j) &= \operatorname*{argmax}_{j'=0}^N v_{i-1}(j') \cdot a_{j' j} \cdot b_{j}(w_i)
         & \mbox{for $i > 0$}
\end{align*}

> Notice how similar the back pointer definition is to the Viterbi best path definition. It merely records which state was used in computing the corresponding best path.

<!-- BEGIN QUESTION -->

**Question:** For the sample sentence above ("canners can canned fish"), calculate the five "layers" of the Viterbi algorithm, that is, $v_0$, $v_1$, $v_2$, $v_3$, and $v_4$, and the corresponding back pointers by filling in the tables below. (We've filled in the $v_0$ column for you already, as per the first two lines in the definition of the Viterbi calculation. No $bp_0$ backpointer is needed (or defined).)

> For the back pointer table, you need only provide entries corresponding to cases where $bp_i(j)$ is non-zero.

<!--
BEGIN QUESTION
name: open_response_viterbi_table
manual: true
-->

<!--TODO: Highest probability table -->
|   | tag         | v_0 &lt;bos&gt; | v_1 canners  | v_2 can                | v_3 canned                   | v_4 fish                   |
|---|:-----------:|:---------------:|:------------:|:----------------------:|:----------------------------:|:--------------------------:|
| 0 | &lt;bos&gt; |    1            | 0            | 0                      | 0                            | 0                          |
| 1 | N           |    0            | 0.8*0.3=0.24 | 0                      | 0                            | 0.012144*1*0.59=0.00716496 |
| 2 | V           |    0            | 0.2*0=0      | 0.24*0.46*0.5=0.0552   | 0.0744*0.33*0.1=0.0024552    | 0                          |
| 3 | M           |    0            | 0            | 0.24*0.31*1=0.0744     | 0                            | 0                          |
| 4 | P           |    0            | 0            | 0                      | 0                            | 0                          |
| 5 | A           |    0            | 0            | 0                      | 0.0552*0.22*1=0.012144       | 0                          |
| 6 | R           |    0            | 0            | 0                      | 0                            | 0                          |

<!--TODO: Backpointers for best path table -->
|   | tag         | bp_1 canners                   | bp_2 can | bp_3 canned | bp_4 fish |
|---|:-----------:|:------------------------------:|:--------:|:-----------:|:---------:|
| 0 | &lt;bos&gt; | 0                              | -        | -           | -         |
| 1 | N           | 0                              | 1        | -           | 5         |
| 2 | V           | 0                              | 1        | 3           | -         |
| 3 | M           | 0                              | -        | -           | -         |
| 4 | P           | 0                              | -        | -           | -         |
| 5 | A           | 0                              | -        | 2           | -         |
| 6 | R           | 0                              | -        | -           | -         |

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Doing this calculation by hand is painful, but it should make clear what's going on. At each entry $v_i(j)$ in the table, we've calculated the probability of the best path through the trellis from the beginning of the sentence to the current word  $x_i$, starting in the start state and ending in the current state $q_j$. To get the maximum probability of all paths in the trellis for the full sentence ending in any state, we merely look up the maximum value of $v_M(j)$ (recalling that $M$ is the length of the sentence).

**Question:** Based on the tables you filled out above, what is the probability of the best path through the trellis for the sample sentence?

<!--
BEGIN QUESTION
name: best_path_probability
manual: true
-->

In [35]:
best_path_probability = 0.00716496

In [36]:
grader.check("best_path_probability")

All tests passed!

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question:** Based on the tables you filled out above, what is the best label sequence for the sentence? Provide it as a list of strings for the variable `best_label_sequence`, e.g., `['<bos>','V','A','M','A']`. (No, that's not the actual answer.)

> Hint: You'll get this by tracing back the backpointers.

<!--
BEGIN QUESTION
name: best_label_sequence
manual: true
-->

In [37]:
best_label_sequence = ["<bos>", "N", "V", "A", "N"]

In [38]:
grader.check("best_label_sequence")

All tests passed!

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question:** What is the complexity of filling in all of the entries in the Viterbi table? How does that compare with the complexity of the total number of paths through the trellis that you calculated above?
<!--
BEGIN QUESTION
name: open_response_how_many_paths_viterbi
manual: true
-->

**Answer**: *For every calculation of the i-th iteration, we need to calculate the max over all the outputs of the (i-1)-th calculation, that is calculating the maximum over N outputs ($O(N)$). That means that each iteration (word) takes $O(N^2)$ steps calculate. For a sequence of $M$ tokens, that complexity is thus $O(M \cdot N^2)$, which is better than the overall number of possible paths*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

# Lab debrief

**Question:** We're interested in any thoughts your group has about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on include the following: 

* Was the lab too long or too short?
* Were the readings appropriate for the lab? 
* Was it clear (at least after you completed the lab) what the points of the exercises were? 
* Are there additions or changes you think would make the lab better?

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# End of lab 2-4

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [39]:
grader.check_all()

best_label_sequence:

    All tests passed!
    

best_path_probability:

    All tests passed!
    

example_highest_label_accuracy:

    All tests passed!
    

example_highest_labeling:

    All tests passed!
    

example_maj_bigram_label_accuracy:

    All tests passed!
    

example_maj_label_accuracy:

    All tests passed!
    

example_majority_bigram_labeling:

    All tests passed!
    

example_majority_labeling:

    All tests passed!
    

highlight1_path_probability:

    All tests passed!
    

highlight2_path_probability:

    All tests passed!
    

majority_path:

    All tests passed!
    

majority_path_probability:

    All tests passed!
    

text_pos:

    All tests passed!